# PoSME CUDA Latency Benchmark
**CPU (DRAM) vs GPU (HBM) pointer-chase comparison**

This measures the actual ASIC resistance ratio by running the
PoSME pointer-chase loop on both system DRAM (CPU) and GPU HBM.

**Requirements:** GPU runtime (T4, A100, or H100).
Go to Runtime > Change runtime type > select GPU.

In [ ]:
# Check GPU
!nvidia-smi --query-gpu=name,memory.total --format=csv,noheader

In [ ]:
# Write CUDA source (embedded)
source = "// PoSME CUDA Benchmark\n// =====================\n// Runs the pointer-chase loop on GPU HBM to measure actual\n// HBM random-access latency vs system DRAM.\n//\n// This is the definitive ASIC resistance test: if the pointer-chase\n// on HBM is only ~2x faster than on DDR5, the latency-bound claim holds.\n//\n// BUILD:  nvcc -O3 posme-cuda-bench.cu -o posme-cuda-bench\n// RUN:    ./posme-cuda-bench\n\n#include <stdio.h>\n#include <stdlib.h>\n#include <string.h>\n#include <time.h>\n\n#define LAMBDA 32\n#define BLOCK_SIZE (2 * LAMBDA)\n\nstruct Block {\n    unsigned char data[LAMBDA];\n    unsigned char causal[LAMBDA];\n};\n\n// Simple hash: SipHash-like mixing for portability.\n// Not cryptographically strong; sufficient for latency measurement.\n__host__ __device__\nstatic void fast_hash(unsigned char out[LAMBDA],\n                      const unsigned char *in, int in_len) {\n    unsigned long long v0 = 0x736f6d6570736575ULL;\n    unsigned long long v1 = 0x646f72616e646f6dULL;\n    unsigned long long v2 = 0x6c7967656e657261ULL;\n    unsigned long long v3 = 0x7465646279746573ULL;\n\n    for (int i = 0; i < in_len; i++) {\n        v3 ^= (unsigned long long)in[i];\n        for (int r = 0; r < 2; r++) {\n            v0 += v1; v2 += v3;\n            v1 = (v1 << 13) | (v1 >> 51); v3 = (v3 << 16) | (v3 >> 48);\n            v1 ^= v0; v3 ^= v2;\n            v0 = (v0 << 32) | (v0 >> 32);\n            v2 += v1; v0 += v3;\n            v1 = (v1 << 17) | (v1 >> 47); v3 = (v3 << 21) | (v3 >> 43);\n            v1 ^= v2; v3 ^= v0;\n            v2 = (v2 << 32) | (v2 >> 32);\n        }\n        v0 ^= (unsigned long long)in[i];\n    }\n\n    v2 ^= 0xff;\n    for (int r = 0; r < 4; r++) {\n        v0 += v1; v2 += v3;\n        v1 = (v1 << 13) | (v1 >> 51); v3 = (v3 << 16) | (v3 >> 48);\n        v1 ^= v0; v3 ^= v2;\n        v0 = (v0 << 32) | (v0 >> 32);\n        v2 += v1; v0 += v3;\n        v1 = (v1 << 17) | (v1 >> 47); v3 = (v3 << 21) | (v3 >> 43);\n        v1 ^= v2; v3 ^= v0;\n        v2 = (v2 << 32) | (v2 >> 32);\n    }\n\n    unsigned long long h = v0 ^ v1 ^ v2 ^ v3;\n    for (int i = 0; i < 8; i++) out[i] = (h >> (i*8)) & 0xff;\n\n    v0 ^= 0xee;\n    for (int r = 0; r < 4; r++) {\n        v0 += v1; v2 += v3;\n        v1 = (v1 << 13) | (v1 >> 51); v3 = (v3 << 16) | (v3 >> 48);\n        v1 ^= v0; v3 ^= v2;\n        v0 = (v0 << 32) | (v0 >> 32);\n        v2 += v1; v0 += v3;\n        v1 = (v1 << 17) | (v1 >> 47); v3 = (v3 << 21) | (v3 >> 43);\n        v1 ^= v2; v3 ^= v0;\n        v2 = (v2 << 32) | (v2 >> 32);\n    }\n\n    h = v0 ^ v1 ^ v2 ^ v3;\n    for (int i = 0; i < 8; i++) out[8+i] = (h >> (i*8)) & 0xff;\n\n    v1 ^= 0xdd;\n    for (int r = 0; r < 4; r++) {\n        v0 += v1; v2 += v3;\n        v1 = (v1 << 13) | (v1 >> 51); v3 = (v3 << 16) | (v3 >> 48);\n        v1 ^= v0; v3 ^= v2;\n        v0 = (v0 << 32) | (v0 >> 32);\n        v2 += v1; v0 += v3;\n        v1 = (v1 << 17) | (v1 >> 47); v3 = (v3 << 21) | (v3 >> 43);\n        v1 ^= v2; v3 ^= v0;\n        v2 = (v2 << 32) | (v2 >> 32);\n    }\n\n    h = v0 ^ v1 ^ v2 ^ v3;\n    for (int i = 0; i < 8; i++) out[16+i] = (h >> (i*8)) & 0xff;\n\n    v2 ^= 0xcc;\n    for (int r = 0; r < 4; r++) {\n        v0 += v1; v2 += v3;\n        v1 = (v1 << 13) | (v1 >> 51); v3 = (v3 << 16) | (v3 >> 48);\n        v1 ^= v0; v3 ^= v2;\n        v0 = (v0 << 32) | (v0 >> 32);\n        v2 += v1; v0 += v3;\n        v1 = (v1 << 17) | (v1 >> 47); v3 = (v3 << 21) | (v3 >> 43);\n        v1 ^= v2; v3 ^= v0;\n        v2 = (v2 << 32) | (v2 >> 32);\n    }\n\n    h = v0 ^ v1 ^ v2 ^ v3;\n    for (int i = 0; i < 8; i++) out[24+i] = (h >> (i*8)) & 0xff;\n}\n\n__host__ __device__\nstatic unsigned int addr_from(const unsigned char cursor[LAMBDA], int j, int n) {\n    unsigned char buf[LAMBDA + 4];\n    memcpy(buf, cursor, LAMBDA);\n    buf[LAMBDA] = (j >> 24) & 0xff;\n    buf[LAMBDA+1] = (j >> 16) & 0xff;\n    buf[LAMBDA+2] = (j >> 8) & 0xff;\n    buf[LAMBDA+3] = j & 0xff;\n    unsigned char h[LAMBDA];\n    fast_hash(h, buf, LAMBDA + 4);\n    unsigned int val = ((unsigned int)h[0] << 24) | ((unsigned int)h[1] << 16) |\n                       ((unsigned int)h[2] << 8) | (unsigned int)h[3];\n    return val % n;\n}\n\n// The critical pointer-chase loop, running on GPU with arena in HBM\n__global__\nvoid posme_step_kernel(Block *arena, int n, int d, int K,\n                       unsigned char *transcript_io) {\n    unsigned char cursor[LAMBDA];\n    memcpy(cursor, transcript_io, LAMBDA);\n\n    for (int t = 1; t <= K; t++) {\n        // Pointer-chase reads -- THIS IS THE LATENCY TEST\n        for (int j = 0; j < d; j++) {\n            unsigned int a = addr_from(cursor, j, n);\n            Block val = arena[a];  // <-- HBM random read\n\n            unsigned char inp[LAMBDA + BLOCK_SIZE];\n            memcpy(inp, cursor, LAMBDA);\n            memcpy(inp + LAMBDA, val.data, LAMBDA);\n            memcpy(inp + LAMBDA + LAMBDA, val.causal, LAMBDA);\n            fast_hash(cursor, inp, LAMBDA + BLOCK_SIZE);\n        }\n\n        // Symbiotic write\n        unsigned int w = addr_from(cursor, d, n);\n        Block old = arena[w];\n\n        unsigned char dinp[3 * LAMBDA];\n        memcpy(dinp, old.data, LAMBDA);\n        memcpy(dinp + LAMBDA, cursor, LAMBDA);\n        memcpy(dinp + 2*LAMBDA, old.causal, LAMBDA);\n        fast_hash(arena[w].data, dinp, 3 * LAMBDA);\n\n        unsigned char cinp[2 * LAMBDA + 4];\n        memcpy(cinp, old.causal, LAMBDA);\n        memcpy(cinp + LAMBDA, cursor, LAMBDA);\n        cinp[2*LAMBDA] = (t >> 24) & 0xff;\n        cinp[2*LAMBDA+1] = (t >> 16) & 0xff;\n        cinp[2*LAMBDA+2] = (t >> 8) & 0xff;\n        cinp[2*LAMBDA+3] = t & 0xff;\n        fast_hash(arena[w].causal, cinp, 2 * LAMBDA + 4);\n\n        // Transcript (simplified)\n        unsigned char tinp[LAMBDA + 4 + LAMBDA];\n        memcpy(tinp, cursor, LAMBDA);\n        tinp[LAMBDA] = (t >> 24) & 0xff;\n        tinp[LAMBDA+1] = (t >> 16) & 0xff;\n        tinp[LAMBDA+2] = (t >> 8) & 0xff;\n        tinp[LAMBDA+3] = t & 0xff;\n        memcpy(tinp + LAMBDA + 4, cursor, LAMBDA); // placeholder for root\n        fast_hash(cursor, tinp, LAMBDA + 4 + LAMBDA);\n    }\n\n    memcpy(transcript_io, cursor, LAMBDA);\n}\n\n// CPU version for comparison\nvoid posme_steps_cpu(Block *arena, int n, int d, int K,\n                     unsigned char *transcript) {\n    unsigned char cursor[LAMBDA];\n    memcpy(cursor, transcript, LAMBDA);\n\n    for (int t = 1; t <= K; t++) {\n        for (int j = 0; j < d; j++) {\n            unsigned int a = addr_from(cursor, j, n);\n            Block val = arena[a];\n\n            unsigned char inp[LAMBDA + BLOCK_SIZE];\n            memcpy(inp, cursor, LAMBDA);\n            memcpy(inp + LAMBDA, val.data, LAMBDA);\n            memcpy(inp + LAMBDA + LAMBDA, val.causal, LAMBDA);\n            fast_hash(cursor, inp, LAMBDA + BLOCK_SIZE);\n        }\n\n        unsigned int w = addr_from(cursor, d, n);\n        Block old = arena[w];\n\n        unsigned char dinp[3 * LAMBDA];\n        memcpy(dinp, old.data, LAMBDA);\n        memcpy(dinp + LAMBDA, cursor, LAMBDA);\n        memcpy(dinp + 2*LAMBDA, old.causal, LAMBDA);\n        fast_hash(arena[w].data, dinp, 3 * LAMBDA);\n\n        unsigned char cinp[2 * LAMBDA + 4];\n        memcpy(cinp, old.causal, LAMBDA);\n        memcpy(cinp + LAMBDA, cursor, LAMBDA);\n        cinp[2*LAMBDA] = (t >> 24) & 0xff;\n        cinp[2*LAMBDA+1] = (t >> 16) & 0xff;\n        cinp[2*LAMBDA+2] = (t >> 8) & 0xff;\n        cinp[2*LAMBDA+3] = t & 0xff;\n        fast_hash(arena[w].causal, cinp, 2 * LAMBDA + 4);\n\n        unsigned char tinp[LAMBDA + 4 + LAMBDA];\n        memcpy(tinp, cursor, LAMBDA);\n        tinp[LAMBDA] = (t >> 24) & 0xff;\n        tinp[LAMBDA+1] = (t >> 16) & 0xff;\n        tinp[LAMBDA+2] = (t >> 8) & 0xff;\n        tinp[LAMBDA+3] = t & 0xff;\n        memcpy(tinp + LAMBDA + 4, cursor, LAMBDA);\n        fast_hash(cursor, tinp, LAMBDA + 4 + LAMBDA);\n    }\n\n    memcpy(transcript, cursor, LAMBDA);\n}\n\nvoid init_arena(Block *arena, int n) {\n    for (int i = 0; i < n; i++) {\n        unsigned char seed[8];\n        seed[0] = (i >> 24) & 0xff; seed[1] = (i >> 16) & 0xff;\n        seed[2] = (i >> 8) & 0xff; seed[3] = i & 0xff;\n        seed[4] = 'p'; seed[5] = 'o'; seed[6] = 's'; seed[7] = 'm';\n        fast_hash(arena[i].data, seed, 8);\n        seed[4] = 'c'; seed[5] = 'a'; seed[6] = 'u'; seed[7] = 's';\n        fast_hash(arena[i].causal, seed, 8);\n    }\n}\n\ndouble now_sec() {\n    struct timespec ts;\n    clock_gettime(CLOCK_MONOTONIC, &ts);\n    return ts.tv_sec + ts.tv_nsec * 1e-9;\n}\n\nvoid hex8(const unsigned char *b, char *out) {\n    for (int i = 0; i < 8; i++) sprintf(out + 2*i, \"%02x\", b[i]);\n    out[16] = 0;\n}\n\nint main() {\n    int n = 1 << 20;  // 64 MiB arena (fits in GPU memory)\n    int d = 8;\n    int K = 1 << 16;  // 64K steps\n\n    printf(\"\\n\");\n    printf(\"  \u2554\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2557\\n\");\n    printf(\"  \u2551   PoSME CUDA Latency Benchmark                        \u2551\\n\");\n    printf(\"  \u2551   CPU (DRAM) vs GPU (HBM) pointer-chase comparison     \u2551\\n\");\n    printf(\"  \u2560\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2563\\n\");\n    printf(\"  \u2551  Arena: %d blocks (%d MiB)                       \u2551\\n\", n, n * BLOCK_SIZE / (1<<20));\n    printf(\"  \u2551  Steps: %d    Reads/step: %d                         \u2551\\n\", K, d);\n    printf(\"  \u255a\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u255d\\n\\n\");\n\n    size_t arena_bytes = (size_t)n * sizeof(Block);\n\n    // --- CPU benchmark ---\n    Block *h_arena = (Block*)malloc(arena_bytes);\n    init_arena(h_arena, n);\n\n    unsigned char transcript[LAMBDA];\n    memset(transcript, 0x42, LAMBDA);\n\n    printf(\"  \u2500\u2500 CPU (System DRAM) \u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\\n\");\n    double t0 = now_sec();\n    posme_steps_cpu(h_arena, n, d, K, transcript);\n    double cpu_time = now_sec() - t0;\n    double cpu_ns = cpu_time / K * 1e9;\n\n    char hex[17];\n    hex8(transcript, hex);\n    printf(\"    Steps: %d\\n\", K);\n    printf(\"    Total: %.3f s\\n\", cpu_time);\n    printf(\"    Avg step: %.0f ns\\n\", cpu_ns);\n    printf(\"    T_K: %s...\\n\\n\", hex);\n\n    // --- GPU benchmark ---\n    Block *d_arena;\n    unsigned char *d_transcript;\n    cudaMalloc(&d_arena, arena_bytes);\n    cudaMalloc(&d_transcript, LAMBDA);\n\n    // Re-init arena and transcript\n    init_arena(h_arena, n);\n    cudaMemcpy(d_arena, h_arena, arena_bytes, cudaMemcpyHostToDevice);\n\n    unsigned char gpu_transcript[LAMBDA];\n    memset(gpu_transcript, 0x42, LAMBDA);\n    cudaMemcpy(d_transcript, gpu_transcript, LAMBDA, cudaMemcpyHostToDevice);\n\n    cudaDeviceSynchronize();\n\n    printf(\"  \u2500\u2500 GPU (HBM) \u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\\n\");\n\n    cudaEvent_t start, stop;\n    cudaEventCreate(&start);\n    cudaEventCreate(&stop);\n\n    cudaEventRecord(start);\n    posme_step_kernel<<<1, 1>>>(d_arena, n, d, K, d_transcript);\n    cudaEventRecord(stop);\n    cudaEventSynchronize(stop);\n\n    float gpu_ms;\n    cudaEventElapsedTime(&gpu_ms, start, stop);\n    double gpu_time = gpu_ms / 1000.0;\n    double gpu_ns = gpu_time / K * 1e9;\n\n    cudaMemcpy(gpu_transcript, d_transcript, LAMBDA, cudaMemcpyDeviceToHost);\n    hex8(gpu_transcript, hex);\n\n    printf(\"    Steps: %d\\n\", K);\n    printf(\"    Total: %.3f s\\n\", gpu_time);\n    printf(\"    Avg step: %.0f ns\\n\", gpu_ns);\n    printf(\"    T_K: %s...\\n\\n\", hex);\n\n    // --- Comparison ---\n    double ratio = cpu_ns / gpu_ns;\n\n    printf(\"  \u2500\u2500 ASIC Resistance Result \u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\\n\\n\");\n    printf(\"    CPU step:   %.0f ns (system DRAM)\\n\", cpu_ns);\n    printf(\"    GPU step:   %.0f ns (HBM)\\n\", gpu_ns);\n    printf(\"    Speedup:    %.2fx\\n\\n\", ratio);\n\n    if (ratio < 3.0) {\n        printf(\"    CLAIM HOLDS: GPU (HBM) advantage is %.1fx (< 3x)\\n\", ratio);\n        printf(\"    The latency-bound ASIC resistance claim is confirmed.\\n\");\n        printf(\"    An adversary with HBM gains only %.1fx, not 8-16x.\\n\\n\", ratio);\n    } else if (ratio < 10.0) {\n        printf(\"    MARGINAL: GPU advantage is %.1fx (3-10x range)\\n\", ratio);\n        printf(\"    Higher than expected. May indicate cache effects or\\n\");\n        printf(\"    hash computation dominating on slow GPU cores.\\n\\n\");\n    } else {\n        printf(\"    CLAIM BROKEN: GPU advantage is %.1fx (>= 10x)\\n\", ratio);\n        printf(\"    The construction is NOT latency-dominated.\\n\\n\");\n    }\n\n    cudaFree(d_arena);\n    cudaFree(d_transcript);\n    free(h_arena);\n\n    return 0;\n}\n"
with open('posme-cuda-bench.cu', 'w') as f:
    f.write(source)
print(f'Wrote {len(source)} bytes')

In [ ]:
# Compile and run
!nvcc -O3 posme-cuda-bench.cu -o posme-cuda-bench 2>&1
!./posme-cuda-bench

In [ ]:
# Hardware details
!echo '=== CPU ==='
!lscpu | grep 'Model name'
!echo '=== Memory ==='
!free -h | head -3
!echo '=== GPU ==='
!nvidia-smi